In [1]:
from pyomo.environ import *

### Example 1: 

- maximize 40x + 30y
- x <= 40
- x + y <= 80
- 2x + y <= 100
- x and y are production rates (units per week)


##### Create model object

In [2]:
#Concrete is when we know everything we need to know right now
m = ConcreteModel()

In [3]:
type(m)

pyomo.core.base.PyomoModel.ConcreteModel

##### Add model variables - with constraints

In [4]:
help(Var)

Help on class Var in module pyomo.core.base.var:

class Var(pyomo.core.base.indexed_component.IndexedComponent, pyomo.core.base.indexed_component.IndexedComponent_NDArrayMixin)
 |  Var(*args, **kwargs)
 |  
 |  A numeric variable, which may be defined over an index.
 |  
 |  Args:
 |      domain (Set or function, optional): A Set that defines valid
 |          values for the variable (e.g., `Reals`, `NonNegativeReals`,
 |          `Binary`), or a rule that returns Sets.  Defaults to `Reals`.
 |      within (Set or function, optional): An alias for `domain`.
 |      bounds (tuple or function, optional): A tuple of (lower, upper)
 |          bounds for the variable, or a rule that returns tuples.
 |          Defaults to (None, None).
 |      initialize (float or function, optional): The initial value for
 |          the variable, or a rule that returns initial values.
 |      rule (float or function, optional): An alias for `initialize`.
 |      dense (bool, optional): Instantiate all el

In [5]:
#Since these are production rates, both variables need to be non-negative.
#  With Var, can add bounds, or common domain constraints (nonnegative, reals, binary)
#Create x and y variables
m.x = Var(domain=NonNegativeReals)
m.y = Var(within=NonNegativeReals) #within is the same as domain

In [6]:
type(m.x)

pyomo.core.base.var.ScalarVar

##### Create Objective
This is our goal: maximizing profit 40x + 30y

In [7]:
#Profit is just the name of our objective.
# note that we use our model variables in the expression
# sense can be minimize or maximize
m.profit = Objective(expr = 40*m.x + 30*m.y, sense=maximize)

In [8]:
type(m.profit)

pyomo.core.base.objective.ScalarObjective

##### Add additional constraints:

- x <= 40
- x + y <= 80
- 2x + y <= 100

In [9]:
#Each of these can be named based off their real-world representation

#demand for product X is less than 40
m.xdemand = Constraint(expr = m.x <= 40)

#labor contraints for producing x and y
m.laborA = Constraint(expr = m.x + m.y <=80)
m.laborB = Constraint(expr = 2*m.x + m.y <=100)

##### Create solver object and solve

Solver notes:
- glpk. "GNU Linear Programming Kit" is a software package written in highly portable C for the solution of mixed integer linear programming and related problems.
- ipopt. "Interior Point Optimizer" for large scale nonlinear optimization of continuous systems.
- cbc. "COIN-OR Branch and Cut" is a mixed integer linear programming solver written in C++. It generally solves the same problems as glpk, but can run multiple threads, and is often faster and more robust.

In [10]:
#I've installed two solvers so far:
print(SolverFactory('glpk').available())
print(SolverFactory('ipopt').available())

True
True


In [11]:
#Create the solver and solve
results = SolverFactory('glpk').solve(m)

In [12]:
type(results)

pyomo.opt.results.results_.SolverResults

##### View the results

In [13]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2600.0
  Upper bound: 2600.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 6
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.015566110610961914
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

In [14]:
if results.solver.status == 'ok': m.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  20.0 :  None : False : False : NonNegativeReals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  60.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*x + 30*y

3 Constraint Declarations
    laborA : Size=1, Index=None, Active=True
        Key  : Lower : Body  : Upper : Active
        None :  -Inf : x + y :  80.0 :   True
    laborB : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x + y : 100.0 :   True
    xdemand : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :    x :  40.0 :   True

6 Declarations: x y profit xdemand laborA laborB


### Notes:

- in the Var Declariations, we can see the value it found for x and y
- Note that in this second print (pprint()), we don't see the profit, just the solved vairables
- Looking at the write() outputs, we see the maximized profit listed under the problem 'bounds'.  (2600)
- Constraint declarations have lower bounds of -Inf. I wonder if adding a discrete negative bound in those constraints would improve processing time.
- **NOTE: This problem is 'neat'.  No integer constraints are included in this example even though all variables resulted in integers.**  Integer constraints should be included; I'll work with that in another example.
